# Tutorial: Building Interactive map using python and Jupyter notebook 

## IP Address locator tool based on the "IP2Location™ LITE IP-COUNTRY Database

### This tutorial includes:
* IP2Location LITE data available from <a href="https://lite.ip2location.com"> https://lite.ip2location.com</a>.
* Leafmap , a Python package for geospatial analysis and interactive mapping in a Jupyter environment (https://leafmap.org/). Wu, Q. (2021). Leafmap: A Python package for interactive mapping and geospatial analysis with minimal coding in a Jupyter environment. Journal of Open Source Software, 6(63), 3414. https://doi.org/10.21105/joss.03414

### How it works

* First, convert the IP address to IP number format. 
* Next, parse the IP address and convert the IP address to an IP number in decimal format to speed up the database query.
* Search IP2Location™ geolocation database (reverse lookup) using IP number to match a record that has the IP Number between the Beginning IP Number (IPFROM) and the Ending IP Number (IPTO)
* Then display on interactive map using the latitude and longitude retrieved from the database.

### IP4 address to IP number conversion

IP address (IPV4) is divided into 4 sub-blocks. Each sub-block has a different weight number each powered by 256. IP number is being used in the database because it is more efficient to search between a range of numbers in a database.

The Beginning IP number and Ending IP Number are calculated based on the following formula:

IP Number = w*256^3 + x*256^2 + y*256 + z  =  w*16777216 + x*65536 + y*256 + z     (1)

where: IP Address = w.x.y.z

Suppose the IP address is "202.186.13.4", then its IP Number will be "3401190660", based on the formula (1).

IP Address = 202.186.13.4

So, w = 202, x = 186, y = 13 and z = 4

IP Number =202*16777216 + 186*65536 + 13*256 + 4
	>>	  =3388997632 +12189696 +3328 +4
	>>	  =3401190660    

#### Import the necessary python packages

In [ ]:
import os
import leafmap.leafmap as leafmap
import pandas as pd 


#### Load the IP-COUNTRY geolocation database of US into panda dataframe and investigate the data. The file is in CSV format.

In [ ]:
in_csv_ip = '../data/US_IP.csv'

dfip = leafmap.csv_to_pandas(in_csv_ip, dtype={"IPFROM": int, "IPTO": int, "COUNTRYCODE": "string", "COUNTRY": "string", "REGION": "string", "CITY": "string", "LATITUDE": float, "LONGITUDE": float, "ZIPCODE": "string"})
dfip. head(5)

#### Check the columns and their datatypes

In [ ]:
dfip.dtypes

#### Prompt for the IP address to geolocate. 

In [ ]:
theip = input("Enter the ip address: ")
ipparts = theip.split('.')
ipparts


#### Convert IP-adress to the ipnumber through the simple formula specified above

In [ ]:
ipnum = 16777216*int(ipparts[0]) + 65536*int(ipparts[1]) + 256*int(ipparts[2]) + int(ipparts[3])
ipnum

#### query the IP-COUNTRY geolocation database that's been loaded into a panda dataframe.

In [ ]:
result = dfip.query('IPFROM <= @ipnum & IPTO >= @ipnum') 
resultdf = pd.DataFrame(result)
resultdf

#### Save result to a csv file

In [ ]:
resultdf.to_csv('../data/myippoint.csv')
in_csv_pts = '../data/myippoint.csv'


#### Display IP location on a map

In [ ]:
Map = leafmap.Map()
Map = leafmap.Map(height="200px", width="300px")
Map = leafmap.Map(draw_control=False, measure_control=False, fullscreen_control=False, attribution_control=True)
Map.add_xy_data(in_csv_pts, x="LONGITUDE", y="LATITUDE", layer_name="ippoint")
Map

#### Read a shapefile of US states and overlay on the map

In [ ]:
in_shp = '../data/us-states.shp'
Map.add_shp(in_shp, layer_name="Zips", fill_colors=['red', 'yellow', 'green', 'orange'])

#### Change the base map to Google's sattelite "Hybrid"

In [ ]:
Map.add_basemap("HYBRID")

#### Another map demo: Create a world population heat map from World Cities data.

In [ ]:
Map2 = leafmap.Map()
in_csv_wc = '../data/world_cities.csv'
dfwc = leafmap.csv_to_pandas(in_csv_wc)
dfwc.dtypes

In [ ]:
Map2.add_heatmap(in_csv_wc, latitude="latitude", longitude='longitude', value="pop_max", name="Heat map", radius=20)
colors = ['blue', 'lime', 'red']
vmin = 0
vmax = 10000
Map2.add_colorbar(colors=colors, vmin=vmin, vmax=vmax)
Map2